<a id="toc"></a>
# HERMES beamline simulation with PyOptiX
***

Contents :
1. [Optical elements declaration](#def_opt)
1. [Definition of optical parameters](#def_param)
1. [Alignment scripts](#alignement)
1. [Execution de la simulation](#exec)
1. [Visualisation](#visu)
1. [Monochromator resolution](#monores)

In [1]:
__author__ = ['Rafael Celestre']
__contact__ = 'rafael.celestre@synchrotron-soleil.fr'
__license__ = 'GPL-3.0'
__copyright__ = 'Synchrotron SOLEIL, Saint Aubin, France'
__created__ = '23/JUL/2024'
__changed__ = '23/JUL/2024'

import ctypes

import numpy as np
import pandas as pd
import pyoptix
import pyoptix.classes as px
from pyoptix.ui_objects import plot_spd_plotly
from scipy.constants import c, degree, eV, h, micro, milli, nano, pi, pico
from typing import List

hc = h*c/eV

pyoptix.set_aperture_active(False)
pyoptix.output_notebook()
# benchmarking tools
# %load_ext autoreload
# %autoreload 2
# %matplotlib widget

INFO:pyoptix.exposed_functions:initializing optix library
INFO:pyoptix.exposed_functions:optix loaded, library version: SR_Source library Alpha release 2.3.740.740 build 2023-05-10



OptiX library initialized


Loading BokehJS ...

<a id="def_opt"></a>
## Optical elements declaration
[Back to the top](#toc)

In [2]:
# beamline class
Hermes = px.Beamline(name="Hermes - current")

# ------------------
# source
# ------------------
ondulator_LE = px.UndulatorSource(name="ond_LE")         # low energy 
ondulator_HE = px.UndulatorSource(name="ond_HE")         # high energy

# ------------------
# optical elements
# ------------------
pupil = px.PlaneFilm(name="pupil")

# M1
m1a = px.PlaneMirror(name="M1A")
m1b = px.ToroidalMirror(name="M1B")
m1c = px.ToroidalMirror(name="M1C")

# monochromator
mono_entrance_slit = px.PlaneFilm(name="foc_hor")
grating_450 = px.PlaneHoloGrating(name="grating_450")
grating_600 = px.PlaneHoloGrating(name="grating_600")
m2 = px.PlaneMirror(name="M2")
m3 = px.ToroidalMirror(name="M3")
mono_exit_slit = px.PlaneFilm(name="foc_ver")

# branch STXM
m4 = px.ToroidalMirror(name="M4")
vs_m4 = px.PlaneFilm(name="virtual_source_M4")

# branch PEEM
m5 = px.CylindricalMirror(name="M5")
vs_m5 = px.PlaneFilm(name="virtual_source_M5")
diagnostic = px.PlaneFilm(name="peem_diagnostic")
m6 = px.ConicCylindricalMirror(name="M6")
m7 = px.ConicCylindricalMirror(name="M7")

# ------------------
# endstations
# ------------------
peem = px.PlaneFilm(name="PEEM")
stxm = px.PlaneFilm(name="STXM")

In [3]:
branches = [[m5, vs_m5, diagnostic, m6, m7, peem],
            [m4, vs_m4, stxm]]

for undulator, m1bis, grating in [(ondulator_LE, m1b, grating_450), (ondulator_HE, m1c, grating_600)]:
    for branch in branches:
        chain_name = f"{undulator.name.split('_')[1]}_G{grating.name.split('_')[1]}_{branch[-1].name}"
        Hermes.chains[chain_name] = [
            undulator, pupil, m1a, m1bis, mono_entrance_slit, grating, m2, m3,
            mono_exit_slit
        ] + branch

for chain_name in Hermes.chains:
    print(chain_name,":\n\t",Hermes.chains[chain_name])

LE_G450_PEEM :
	 ond_LE -> pupil -> M1A -> M1B -> foc_hor -> grating_450 -> M2 -> M3 -> foc_ver -> M5 -> virtual_source_M5 -> peem_diagnostic -> M6 -> M7 -> PEEM 
LE_G450_STXM :
	 ond_LE -> pupil -> M1A -> M1B -> foc_hor -> grating_450 -> M2 -> M3 -> foc_ver -> M4 -> virtual_source_M4 -> STXM 
HE_G600_PEEM :
	 ond_HE -> pupil -> M1A -> M1C -> foc_hor -> grating_600 -> M2 -> M3 -> foc_ver -> M5 -> virtual_source_M5 -> peem_diagnostic -> M6 -> M7 -> PEEM 
HE_G600_STXM :
	 ond_HE -> pupil -> M1A -> M1C -> foc_hor -> grating_600 -> M2 -> M3 -> foc_ver -> M4 -> virtual_source_M4 -> STXM 


<a id="def_param"></a>
## Definition of optical parameters

In this section we define the static parameters of the optical elements. Characteristics that change with 
either configuration or energy are defined in the [Alignment scripts](#alignement) section.

[Back to the top](#toc)

In [4]:
number_of_rays = 5000

### Low energy undulator (**HU64**)

In [5]:
e_beam = px.ElectronBeam()
e_beam.from_twiss(energy=2.75, energy_spread=0.1025/100, current=0.500,
                  emittance=3.94*nano, coupling=1/100,
                  beta_x=4.7890, eta_x= 0.1804, etap_x= 0.0007, alpha_x=-0.3858,
                  beta_y=3.7497, eta_y=-0.0044, etap_y=-0.0025, alpha_y=-0.7746)
e_beam.print_rms()

hu64 = px.MagneticStructure(period_length=64e-3, number_of_periods=28)

ondulator_LE.electron_beam = e_beam
ondulator_LE.magnetic_structure = hu64

ondulator_LE.write_syned_config(r".\resources\soleil_hu64","Soleil - HU64")

electron beam:
              >> x/xp = 229.94 um vs. 30.60 urad
              >> y/yp = 12.91 um vs. 4.82 urad


### High energy undulator (**HU42**)

In [6]:
e_beam = px.ElectronBeam()
e_beam.from_twiss(energy=2.75, energy_spread=0.1025/100, current=0.500,
                  emittance=3.94*nano, coupling=1/100,
                  beta_x=4.1825, eta_x=0.1792, etap_x= 0.0007, alpha_x=0.0578,
                  beta_y=2.3439, eta_y=0.0001, etap_y=-0.0025, alpha_y=0.0143)
e_beam.print_rms()

hu42 = px.MagneticStructure(period_length=42e-3, number_of_periods=42)

ondulator_HE.electron_beam = e_beam
ondulator_HE.magnetic_structure = hu42

ondulator_HE.write_syned_config(r".\resources\soleil_hu42","Soleil - HU42")

electron beam:
              >> x/xp = 223.73 um vs. 30.60 urad
              >> y/yp = 9.56 um vs. 4.82 urad


### Entrance pupil
[Back to the top](#toc)

In [7]:
pupil.distance_from_previous = 20
pupil.recording_mode = px.RecordingMode.recording_output
pupil.next = m1a

### M1

[Back to the top](#toc)

In [8]:
m1_grazing_angle = [2.5*degree, 1.2*degree]

m1_distances = px.M1_triad_distances(incident_angle=m1_grazing_angle,
                                     transverse_distance=41e-3, 
                                     verbose=1)

Distance M1B to M1A: 470.422 mm
- proj. distance into the opt. axis: 468.632 mm

Distance M1C to M1A: 979.089 mm
- proj. distance into the opt. axis: 978.230 mm



#### - M1A

In [9]:
m1a.distance_from_previous = 0
m1a.phi = -90*degree # rad
m1a.recording_mode = px.RecordingMode.recording_output

#### - M1B

In [10]:
m1b.distance_from_previous = m1_distances[0][0]
m1b.theta = m1_grazing_angle[0]
m1b.phi = 180*degree   
m1b.minor_curvature = 1/1.802572 # m-1
m1b.major_curvature = 1/126.7433 # m-1
m1b.recording_mode = px.RecordingMode.recording_output
m1b.next = mono_entrance_slit

#### - M1C

In [11]:
m1c.distance_from_previous = m1_distances[1][0]
m1c.theta = m1_grazing_angle[1] 
m1c.phi = 180*degree   
m1c.minor_curvature = 1/0.8870043 # m-1
m1c.major_curvature = 1/229.5203 # m-1
m1c.recording_mode = px.RecordingMode.recording_output
m1c.next = mono_entrance_slit

### Monochromator

[Back to the top](#toc)

#### Mono entrance slit

In [12]:
mono_entrance_slit.phi = -90*degree
mono_entrance_slit.recording_mode = px.RecordingMode.recording_output

#### Grating 450 l/mm

In [18]:
grating_450.distance_from_previous = 0.6
grating_450.line_density = 450/milli
grating_450.inverse_distance1=-0.1761099
grating_450.inverse_distance2=-0.1111111
grating_450.elevation_angle1=-min(np.arccos(0.7986355-0.157995),np.arccos(0.7986355))
grating_450.recording_wavelength=351.1*nano
grating_450.show_vls_law(100e-3, 3)
grating_450.order_align = 1
grating_450.order_use = 1
grating_450.recording_mode = px.RecordingMode.recording_output
grating_450.next = m2

C hologram info for grating_450: 
	 groove/m = 450000.0094938453 + -4913.516826031962 * x + -18413.013775072992 * x^2 + 4956.014995574951 * x^3
	 line radius curvature = 2.4307455744618998
	 line tilt = 0.0


#### Grating 600 l/mm

In [19]:
grating_600.distance_from_previous = 0.6
grating_600.line_density = 600/milli
grating_600.inverse_distance1=-0.2002547
grating_600.inverse_distance2=-0.1111111
grating_600.elevation_angle1=-min(np.arccos(0.7986355), np.arccos(0.7986355-0.21066))
grating_600.recording_wavelength=351.1*nano
grating_600.show_vls_law(100e-3, 3)
grating_600.order_align = 1
grating_600.order_use = 1
grating_600.recording_mode = px.RecordingMode.recording_output
grating_600.next = m2

C hologram info for grating_600: 
	 groove/m = 600000.0163698791 + -483.5964154418325 * x + -29279.683600246906 * x^2 + 8137.222845494747 * x^3
	 line radius curvature = 2.363162423410606
	 line tilt = 0.0


#### M2

In [ ]:
m2.phi = 180*degree 
m2.recording_mode = px.RecordingMode
m2.next = m3

#### M3

In [20]:
m3.theta = 1.2*degree
m3.phi = -90*degree   
m3.minor_curvature = 1/0.1462124
m3.major_curvature = 1/83
m3.recording_mode = px.RecordingMode.recording_output
m3.next = mono_exit_slit

#### Mono exit sit